# Guide: how to run Qiskit in distributed fashion

In [2]:
from typing import List

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.random import random_circuit

from quantum_serverless import QuantumServerless, run_qiskit_remote, get, put

First, we need quantum serverless object to control where our code will be executed

Let's also create some circuit to use for examples

In [7]:
serverless = QuantumServerless()

serverless

<QuantumServerless: providers [local], clusters [local]>

In [8]:
circuit = random_circuit(5, 3)

circuit.draw()

┌───┐┌─────┐               ┌────────────┐     
q_0: ──────────────┤ X ├┤ Tdg ├───────────────┤ Rz(1.9799) ├─────
     ┌────────────┐└─┬─┘└─────┘               └─────┬──────┘     
q_1: ┤ Ry(4.1801) ├──┼─────■────────────────────────┼─────────■──
     └───┬───┬────┘  │   ┌─┴─┐                      │       ┌─┴─┐
q_2: ────┤ Y ├───────┼───┤ X ├──────────────────────┼───────┤ X ├
         └───┘       │   └─┬─┘ ┌─────────────┐      │       └─┬─┘
q_3: ────────────────■─────┼───┤ Ry(0.94329) ├──────■─────────┼──
                     │     │   └─────────────┘                │  
q_4: ────────────────■─────■──────────────────────────────────■──

In order to handle remote compute we need to be able to turn objects and functions into remote functions and objects.

`put` function allows to make your local object into remote object stored in distributed manner in shared memory.
As result you get reference to remote object, which you can use to retrieve it back

Example:
```python
remote_object_reference = put(local_object)
```

`get` function allows you to get remote objects back to your local machine by reference

Example:
```python
local_object = put(remote_object_reference)
```

Let's see how we can turn our locally generated circuit into remote object and back

In [9]:
with serverless:
    # make any object as remote object
    circuit_reference = put(circuit)
    print("Circuit reference:", circuit_reference)
    
    # to get any object from remote to local call `get` function
    retrieved_object = get(circuit_reference)
    print(f"Retrieved object\n{retrieved_object}")

Circuit reference: ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000003000000)
Retrieved object
                   ┌───┐┌─────┐               ┌────────────┐     
q_0: ──────────────┤ X ├┤ Tdg ├───────────────┤ Rz(1.9799) ├─────
     ┌────────────┐└─┬─┘└─────┘               └─────┬──────┘     
q_1: ┤ Ry(4.1801) ├──┼─────■────────────────────────┼─────────■──
     └───┬───┬────┘  │   ┌─┴─┐                      │       ┌─┴─┐
q_2: ────┤ Y ├───────┼───┤ X ├──────────────────────┼───────┤ X ├
         └───┘       │   └─┬─┘ ┌─────────────┐      │       └─┬─┘
q_3: ────────────────■─────┼───┤ Ry(0.94329) ├──────■─────────┼──
                     │     │   └─────────────┘                │  
q_4: ────────────────■─────■──────────────────────────────────■──
                                                                 


As you can see we made our circuit into remote object and get it back within serverless context

Now, let's see how we can do the same thing with functions.

First, we will create more example object to play around with

In [10]:
circuit_batches = [
    [random_circuit(5, 3) for _ in range(2)]
    for n_batch in range(3)
]
circuit_batches

[[<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7f7fa895c2d0>,

`run_qiskit_remote` decorator is used to turn any function into remote function.
So, when next time you will call this function it will be executed as remote function and you will get reference to this function back.

Let's create remote transpilation procedure to transpile our circuits remotely.

First, we annotate our `remote_transpile` function with `run_qiskit_remote` decorator.
Then we run our function, get back function reference and then fetch result of function by reference

In [11]:
@run_qiskit_remote()
def remote_transpile(circuits: List[QuantumCircuit]):
    return transpile(circuits)

with serverless.context():
    remote_function_reference = remote_transpile(circuit_batches[0])
    print("Function reference:", remote_function_reference)
    
    result = get(remote_function_reference)
    print("Function result:", result)

Function reference: ObjectRef(c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000)
Function result: [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f7fa892c950>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f7f68391a90>]


Because our functions are running as remote async functions, we can run multiple of them in parallel.
in order to do that, you just call them in a loop and immideatly get references.

You can run `get` on those reference and you will get back all results back to local machine

In [12]:
with serverless:
    remote_function_references = [
        remote_transpile(circuit_batch)
        for circuit_batch in circuit_batches
    ]
    print("Functions references:", remote_function_references)
    
    result = get(remote_function_references)
    print("Functions resulst:", result)

Functions references: [ObjectRef(16310a0f0a45af5cffffffffffffffffffffffff0100000001000000), ObjectRef(c2668a65bda616c1ffffffffffffffffffffffff0100000001000000), ObjectRef(32d950ec0ccf9d2affffffffffffffffffffffff0100000001000000)]
Functions resulst: [[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f7f68391810>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f7f98d75f50>], [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f7fa895c050>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f7fa892c050>], [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f7fca66f550>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f7f78488290>]]
